In [1]:
#import the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [150]:
df_train = pd.read_csv('train.csv')  #TRAIN DATA WITH LABELS
print(df_train.shape)
print(df_train.head(2))

(891, 12)
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   

   Parch     Ticket     Fare Cabin Embarked  
0      0  A/5 21171   7.2500   NaN        S  
1      0   PC 17599  71.2833   C85        C  


In [151]:
df_test = pd.read_csv('test.csv')   #UNLABELED TEST DATA
print(df_test.shape)
print(df_test.head(2))

(418, 11)
   PassengerId  Pclass                              Name     Sex   Age  SibSp  \
0          892       3                  Kelly, Mr. James    male  34.5      0   
1          893       3  Wilkes, Mrs. James (Ellen Needs)  female  47.0      1   

   Parch  Ticket    Fare Cabin Embarked  
0      0  330911  7.8292   NaN        Q  
1      0  363272  7.0000   NaN        S  


'Survived' attribute is missing from test.csv


In [152]:
df_gender = pd.read_csv('gender_submission.csv')   # LABELLED TEST DATA

In [153]:
print(df_gender.head(2))

   PassengerId  Survived
0          892         0
1          893         1


In [154]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [155]:
df_test = pd.merge(df_test,df_gender , how='left', on=['PassengerId'])  
df_test.tail() 

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,0
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C,0


In [156]:
#find the number of missing values from training data
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [129]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
Survived         0
dtype: int64

In [157]:
df_train.describe()    #survived mean is 38% ... that means 38% of the people survived.

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [131]:
#lets use the corr feature to see which feature contributes the most to the survival rate
df_train.corr()["Survived"].sort_values()

#we see that Pclass has the highest negative correlation with survival rate. 

Pclass        -0.338481
Age           -0.077221
SibSp         -0.035322
PassengerId   -0.005007
Parch          0.081629
Fare           0.257307
Survived       1.000000
Name: Survived, dtype: float64

In [158]:
#changing data to numeric
#let female = 0 and male = 1

df_train["Sex"].loc[df_train["Sex"]=='female'] = 0
df_train["Sex"].loc[df_train["Sex"] == 'male'] = 1

# converting the age 0-15yr as 0 and 15-80 as 1,ie 1 is older and 0 is young.
df_train['AgeRange'] = pd.cut(df_train['Age'], [0, 15, 80], labels=[0, 1])

df_train['FamilyCount'] = (df_train['SibSp'] + df_train['Parch'])  #total family count 

# 1 if parch > 0 or sibiling > 0 i.e if passenger is travelling alone it will be 0
df_train['FamilyBool'] = (df_train['SibSp'] > 0) | (df_train['Parch'] > 0)

# 1 if parch > 0
df_train['ParchBool'] = (df_train['Parch'] > 0)

# 1 sibiling > 0
df_train['SibSpBool'] = (df_train['SibSp'] > 0)

df_train.tail()


/Users/gayatri/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeRange,FamilyCount,FamilyBool,ParchBool,SibSpBool
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.00,NaN,S,1.0,0,False,False,False
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0,112053,30.00,B42,S,1.0,0,False,False,False
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,NaN,1,2,W./C. 6607,23.45,NaN,S,NaN,3,True,True,True
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.00,C148,C,1.0,0,False,False,False
890,891,0,3,"Dooley, Mr. Patrick",1,32.0,0,0,370376,7.75,NaN,Q,1.0,0,False,False,False


In [160]:
#### Lets do the same for the test set

df_test["Sex"].loc[df_test['Sex'] == 'female'] = 0
df_test["Sex"].loc[df_test['Sex'] == 'male'] = 1
df_test["FamilyCount"] = df_test["SibSp"] + df_test["Parch"]
df_test["FamilyBool"] = (df_test["SibSp"] > 0) | (df_test["Parch"] >0)
df_test["ParchBool"] = (df_test["Parch"] > 0)
df_test["SibSpBool"] = (df_test["SibSp"] >0)
df_test.tail()


/Users/gayatri/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived,FamilyCount,FamilyBool,ParchBool,SibSpBool
413,1305,3,"Spector, Mr. Woolf",1,NaN,0,0,A.5. 3236,8.0500,NaN,S,0,0,False,False,False
414,1306,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,108.9000,C105,C,1,0,False,False,False
415,1307,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0,0,False,False,False
416,1308,3,"Ware, Mr. Frederick",1,NaN,0,0,359309,8.0500,NaN,S,0,0,False,False,False
417,1309,3,"Peter, Master. Michael J",1,NaN,1,1,2668,22.3583,NaN,C,0,2,True,True,True


In [161]:
#Let us drop missing values.
df_train_clean_age = df_train.dropna(subset=["Age"])


In [162]:
survived_class = df_train.groupby('Pclass')['Survived'].mean()
survived_class
#the better the class better survival rate!!!

Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64

In [135]:
#let us check the gender now!
survived_gender= df_train.groupby('Sex')['Survived'].mean()
survived_gender

#Females have higher survival rate!!

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

In [49]:
#let us check the age group now

survived_age = df_train.groupby('AgeRange')['Survived'].mean()
survived_age

#children have a higher survival rate!!!! (age 0 -15)

AgeRange
0    0.590361
1    0.381933
Name: Survived, dtype: float64

In [163]:
#let us now neglect under correlated features and only consider important ones

features = ['Pclass','Sex','Fare','FamilyBool','ParchBool','SibSpBool']
cols = np.append(['Survived'],features)
print(cols)
print(df_train.head(2))
print(df_train.shape)

['Survived' 'Pclass' 'Sex' 'Fare' 'FamilyBool' 'ParchBool' 'SibSpBool']
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   

                                                Name  Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris    1  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    0  38.0      1      0   

      Ticket     Fare Cabin Embarked AgeRange  FamilyCount  FamilyBool  \
0  A/5 21171   7.2500   NaN        S        1            1        True   
1   PC 17599  71.2833   C85        C        1            1        True   

   ParchBool  SibSpBool  
0      False       True  
1      False       True  
(891, 17)


In [165]:
print(cols)
print(df_train.head(2))
df_train2 = df_train.loc[:,cols].dropna().reindex()   #remove null values
X = df_train.loc[:,features]
y = np.ravel(df_train.loc[:,['Survived']])
print(X.shape)
print(y.shape)

['Survived' 'Pclass' 'Sex' 'Fare' 'FamilyBool' 'ParchBool' 'SibSpBool']
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   

                                                Name  Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris    1  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    0  38.0      1      0   

      Ticket     Fare Cabin Embarked AgeRange  FamilyCount  FamilyBool  \
0  A/5 21171   7.2500   NaN        S        1            1        True   
1   PC 17599  71.2833   C85        C        1            1        True   

   ParchBool  SibSpBool  
0      False       True  
1      False       True  
(891, 6)
(891,)


In [166]:
df_test21 = df_test.loc[:,cols].dropna().reindex()
print(df_test21.shape)
X_test = df_test21.loc[:,features]
y_test = np.ravel(df_test21.loc[:,['Survived']]) 
print(X_test.shape)
print(y_test.shape)

(417, 7)
(417, 6)
(417,)


In [167]:
#### TRAINING THE MODEL
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import svm

clf_log = LogisticRegression()
clf_log = clf_log.fit(X,y)
y_pred = clf_log.predict(X_test)
print('prediction complete')

prediction complete


In [169]:
ours = accuracy_score(y_test, y_pred) * 100
print('Accuracy! {}'.format(acc_tree))

acc_tree = accuracy_score(y_test, y_pred,normalize=False)
print('We predicted {} out of {} successfully!.'.format(ours,y_test.shape))
print(classification_report(y_test, y_pred))

Accuracy! 412
We predicted 98.8009592326139 out of (417,) successfully!.
             precision    recall  f1-score   support

          0       1.00      0.98      0.99       265
          1       0.97      1.00      0.98       152

avg / total       0.99      0.99      0.99       417

